In [2]:
#라이브러리
import tempfile
import os
import zipfile
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
from tensorflow import keras

In [3]:
# 함수 
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [4]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3013 - accuracy: 0.9177 - val_loss: 0.1250 - val_accuracy: 0.9667
Epoch 2/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1190 - accuracy: 0.9661 - val_loss: 0.0927 - val_accuracy: 0.9740
Epoch 3/4
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0853 - accuracy: 0.9755 - val_loss: 0.0711 - val_accuracy: 0.9803
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0689 - accuracy: 0.9803 - val_loss: 0.0703 - val_accuracy: 0.9813


In [5]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9786999821662903
Saved baseline model to: C:\Users\Jay\AppData\Local\Temp\tmp_nsq32a1.h5


C:\Users\Jay\AppData\Local\Temp\ipykernel_8252\3664931980.py:6: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [6]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_flatte  (None, 2028)              1         
 n (PruneLowMagnitude)                                           
                                                        

In [7]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 5s 9ms/step - loss: 0.0868 - accuracy: 0.9765 - val_loss: 0.0929 - val_accuracy: 0.9770
Epoch 2/2
422/422 [==============================] - 4s 9ms/step - loss: 0.0930 - accuracy: 0.9749 - val_loss: 0.0794 - val_accuracy: 0.9795


In [8]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9786999821662903
Pruned test accuracy: 0.9751999974250793


In [9]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# 프루닝 모델 저장
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: C:\Users\Jay\AppData\Local\Temp\tmpfikzhzio.h5


C:\Users\Jay\AppData\Local\Temp\ipykernel_8252\3008039924.py:5: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


In [11]:
# 파일 크기를 정수로 변환하여 출력
keras_file_size = int(get_gzipped_model_size(keras_file))
pruned_keras_file_size = int(get_gzipped_model_size(pruned_keras_file))

print("Size of gzipped baseline Keras model: %d bytes" % keras_file_size)
print("Size of gzipped pruned Keras model: %d bytes" % pruned_keras_file_size)


Size of gzipped baseline Keras model: 78270 bytes
Size of gzipped pruned Keras model: 25819 bytes


In [ ]:
#압축률 = (압축 전 데이터 크기) / (압축 후 데이터 크기)

pruned_keras_compression_rate = ((keras_file_size / pruned_keras_file_size))
print("compression rate of pruned Keras model: %d" % pruned_keras_compression_rate)


compression rate of pruned Keras model: 3


In [ ]:
# 백분율 정확도 차이(Percentage Accuracy Difference) = [(전체 정확도 - 정확도 차이) / 전체 정확도] * 100%
PAD = (baseline_model_accuracy - (baseline_model_accuracy - model_for_pruning_accuracy)) / baseline_model_accuracy * 100

In [ ]:
PAD

99.04032566103803